In [192]:
from pathlib import Path
import json
import os

from tinydb import TinyDB

current_dir = Path(os.getcwd()).absolute()
results_dir = current_dir.joinpath('results')
kv_data_dir = results_dir.joinpath('kvdb')
kv_data_dir.mkdir(parents=True, exist_ok=True)


def _load_json(json_path):
    with open(json_path) as f:
        return json.load(f)


class DocumentDB(object):
    def __init__(self, db_path):
        ## You can use the code from the previous exmaple if you would like
        people_json = kv_data_dir.joinpath('people.json')
        visited_json = kv_data_dir.joinpath('visited.json')
        sites_json = kv_data_dir.joinpath('sites.json')
        measurements_json = kv_data_dir.joinpath('measurements.json')

        self._db_path = Path(db_path)
        self._db = None
        self._person_lookup = _load_json(people_json)
        #TODO
        self._measurements_lookup = _load_json(measurements_json)
        self._site_lookup = _load_json(sites_json)
        self._visit_lookup = _load_json(visited_json)
        self._load_db()
        
    def _get_site(self, site_id):
        return self._site_lookup[str(site_id)]

    def _get_measurements(self, person_id):
        measurements = []

        for values in self._measurements_lookup.values():
            for key, value in values.items():
                if str(values['person_id']) == str(person_id):
                    measurements.extend([{key: value}])
            #measurements.extend([value for value in values if str(values['person_id']) == str(person_id)])

        return measurements

    def _get_visit(self, visit_id):
        visit = self._visit_lookup.get(str(visit_id))
        print(visit)
        #TODO: 
        site_id = set([visits['site_id'] for visits in visit])
        site = []
        for sites in site_id:
            site = self._get_site(sites)
                
             site['measurements'] = [
                measurement for measurement in measurements
                if sites == visits['site_id']
                ]
            site.append(sites)
            visit['site'] = site

        return visit

    def _load_db(self):
        self._db = TinyDB(self._db_path)
        persons = self._person_lookup.items()
        for person_id, record in persons:
            measurements = self._get_measurements(person_id)
            for index in range(len(measurements)):
                visit_ids =  set([measurements[index]['visit_id']]) #{k:[v for v in measurements[k]] for k in measurements.keys()}
                visits = []
                for visit_id in visit_ids:
                    visit = self._get_visit(visit_id)
                    print(visit)
                    visit['measurements'] = [
                        measurement for measurement in measurements
                        if visit_id == measurement['visit_id']
                        ]
                    visits.append(visit)

                record['visits'] = visits
                self._db.insert(record)

IndentationError: unexpected indent (<ipython-input-192-09234a5a5f13>, line 58)

In [147]:
from pathlib import Path
import json
import os

from tinydb import TinyDB

current_dir = Path(os.getcwd()).absolute()
results_dir = current_dir.joinpath('results')
kv_data_dir = results_dir.joinpath('kvdb')
kv_data_dir.mkdir(parents=True, exist_ok=True)


def _load_json(json_path):
    with open(json_path) as f:
        return json.load(f)


class DocumentDB(object):
    def __init__(self, db_path):
        ## You can use the code from the previous exmaple if you would like
        people_json = kv_data_dir.joinpath('people.json')
        visited_json = kv_data_dir.joinpath('visited.json')
        sites_json = kv_data_dir.joinpath('sites.json')
        measurements_json = kv_data_dir.joinpath('measurements.json')

        self._db_path = Path(db_path)
        self._db = None
        self._person_lookup = _load_json(people_json)
        #TODO
        self._measurements_lookup = _load_json(measurements_json)
        self._site_lookup = _load_json(sites_json)
        self._visit_lookup = _load_json(visited_json)
        self._load_db()
        
    def _get_site(self, site_id):
        return self._site_lookup[str(site_id)]

    def _get_measurements(self, person_id):
        measurements = []

        for values in self._measurements_lookup.values():
            if values.get('person_id') == str(person_id):
                measurements.append(values)
            #extend([value for value in values if values.get('person_id') == str(person_id)])

        return measurements

    def _get_visit(self, visit_id):
        visit = self._visit_lookup.values()
        #TODO:
        for visits in visit:
            site_id = set([visits['site_id']])
            site = []

            if str(visits['site_id']) == str(site_id):
                measurements.append(visits['site_id'])
                site.append(visits['site_id'])
            list(visit).append(site)

        return list(visit)

    def _load_db(self):
        self._db = TinyDB(self._db_path)
        persons = self._person_lookup.items()
        for person_id, record in persons:
            measurements = self._get_measurements(person_id)
            visit_ids = set([measurement['visit_id'] for measurement in measurements])
            visits = []
            for visit_id in visit_ids:
                visit = self._get_visit(visit_id)
                for measurement in measurements:
                    if visit_id == measurement['visit_id']:
                        visits.append(visit)

                record['visits'] = visits
                self._db.insert(record)


In [148]:
db_path = results_dir.joinpath('patient-info.json')
if db_path.exists():
    os.remove(db_path)

db = DocumentDB(db_path)